# 🎯 Valentina Facial LoRA Trainer - Google Colab Edition

Este notebook treina uma **LoRA de identidade facial** para a Valentina usando FLUX.1-dev + dataset otimizado no Google Colab.

## 🎯 Foco: Identidade Visual (NÃO NSFW)
- **Objetivo**: Gerar a Valentina com máxima consistência facial
- **Dataset**: Imagens da pasta `valentina_identity_4lora_dataset_flux`
- **Características**: 25 anos, rosto oval, olhos amendoados, sem tatuagens
- **Trigger Word**: `vltna woman`

## 📋 Stack Otimizada para Identidade:
- **Base**: FLUX.1-dev (máxima qualidade facial)
- **Dataset**: 18 imagens com seeds sequenciais (máxima consistência)
- **Parâmetros**: Conservadores para preservar características faciais
- **Output**: LoRA facial da Valentina para uso local com mflux

⚠️ **Importante**: Execute as células em ordem e aguarde a conclusão de cada etapa.

In [ ]:
# --- Configurações Principais ---
BASE_MODEL_ID = "black-forest-labs/FLUX.1-dev" # Modelo base oficial para identidade
DATASET_ZIP_FILENAME = "valentina_identity_4lora_dataset_flux.zip" # Dataset de identidade gerado

# --- Caminhos no Ambiente Colab ---
COLAB_MODELS_PATH = "/content/models"
COLAB_DATASET_PATH = "/content/dataset"
COLAB_OUTPUT_PATH = "/content/output_lora"
COLAB_LOGS_PATH = "/content/logs"

# --- Parâmetros de Treinamento da LoRA de Identidade ---
INSTANCE_PROMPT = "a photo of vltna woman" # Token único para Valentina
CLASS_PROMPT = "a photo of a woman" # Prompt de classe para regularização

# Configurações de resolução e qualidade (baseado no dataset gerado)
RESOLUTION = 1024  # Mesma resolução do dataset
CENTER_CROP = True
RANDOM_FLIP = False # NUNCA flipar para preservar características faciais

# Batch sizes otimizados para identidade (conservadores)
TRAIN_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 4 # Batch efetivo menor para preservar detalhes

# Learning rates CONSERVADORES para preservar identidade facial
LEARNING_RATE = 5e-5 # Reduzido para preservar características faciais
UNET_LR = 5e-5
TEXT_ENCODER_LR = 3e-6 # Muito menor para text encoder

# Scheduler e warmup otimizados para identidade
LR_SCHEDULER = "cosine_with_restarts"
LR_WARMUP_STEPS = 50 # Reduzido para dataset menor
LR_NUM_CYCLES = 1

# Steps de treinamento (calculado para dataset de identidade: ~80-120 steps por imagem)
MAX_TRAIN_STEPS = 1500 # Para 18 imagens = ~83 steps/imagem (conservador)
SAVE_STEPS = 300
VALIDATION_EPOCHS = 3 # Reduzido para dataset menor

# Arquitetura LoRA otimizada para IDENTIDADE FACIAL
LORA_RANK = 64 # Rank menor para preservar identidade (era 128)
LORA_ALPHA = 32 # Alpha = rank/2 para estabilidade
LORA_DROPOUT = 0.05 # Dropout menor para melhor aprendizado

# Configurações de precisão e otimização
MIXED_PRECISION = "bf16" # Melhor qualidade se suportado
USE_8BIT_ADAM = True
ADAM_BETA1 = 0.9
ADAM_BETA2 = 0.999
ADAM_WEIGHT_DECAY = 0.01
ADAM_EPSILON = 1e-8
MAX_GRAD_NORM = 1.0

# Memory optimization
GRADIENT_CHECKPOINTING = True
ENABLE_XFORMERS = True
USE_CPU_OFFLOAD = False # Manter na GPU para velocidade

# Regularização para IDENTIDADE FACIAL
PRIOR_LOSS_WEIGHT = 1.0
SNR_GAMMA = 5.0 # Para melhor qualidade com ruído

# Checkpointing conservador
CHECKPOINTING_STEPS = 300
CHECKPOINTS_TOTAL_LIMIT = 3
RESUME_FROM_CHECKPOINT = None

# Seeds para reprodutibilidade (alinhado com dataset)
SEED = 42 # Mesmo seed base do dataset

# Configurações específicas para IDENTIDADE (não NSFW)
USE_MIDJOURNEY_LORA = False # Desabilitado - foco em identidade pura
VALIDATION_PROMPT = "a photo of vltna woman, professional portrait photography" # Prompt de validação para identidade

print("⚙️ Configurações otimizadas para IDENTIDADE FACIAL carregadas:")
print(f"📊 Steps totais: {MAX_TRAIN_STEPS}")
print(f"🎯 Batch efetivo: {TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"🧠 LoRA Rank: {LORA_RANK} (reduzido para preservar identidade)")
print(f"🎨 Base Model: {BASE_MODEL_ID}")
print(f"💼 Dataset: {DATASET_ZIP_FILENAME}")
print(f"🎭 Trigger: '{INSTANCE_PROMPT}'")
print(f"🔬 Foco: IDENTIDADE FACIAL (não NSFW)")

## Célula 2: Setup do Ambiente Colab (Dependências)

In [ ]:
# 📦 INSTALAÇÃO OTIMIZADA DE DEPENDÊNCIAS VIA UV
print("🔧 Instalando dependências usando uv para maior robustez...")

# 1. Instalar uv
print("⚙️ Instalando uv...")
!pip install -q uv
print("✅ uv instalado.")

# 2. Definir o conteúdo do requirements.txt baseado no pyproject.toml do mflux
requirements_content = """\
torch>=2.1.0,<2.4.0
torchvision>=0.16.0,<0.19.0
torchaudio>=2.1.0,<2.4.0
diffusers[torch]>=0.27.0,<1.0
transformers>=4.44.0,<5.0
accelerate>=0.32.0,<1.0
safetensors>=0.4.0,<0.5.0
xformers>=0.0.27,<0.0.29
pillow>=10.0.0,<11.0.0
opencv-python>=4.10.0,<5.0
huggingface-hub>=0.24.5,<1.0
sentencepiece>=0.2.0,<0.3.0
tokenizers>=0.19.0,<0.20.0
protobuf>=5.27.0,<6.0.0
numpy>=2.0.0,<3.0.0
requests>=2.32.0,<3.0.0
scipy>=1.14.0,<2.0.0
matplotlib>=3.9.0,<4.0.0
omegaconf>=2.3.0,<3.0.0
einops>=0.8.0,<0.9.0
invisible-watermark>=0.2.0,<0.3.0
compel>=2.0.0,<3.0.0
wandb>=0.17.0,<0.18.0
peft>=0.12.0,<0.13.0
bitsandbytes>=0.43.0,<0.44.0
gradio>=4.39.0,<5.0.0
albumentations>=1.4.0,<2.0.0
imageio>=2.34.0,<3.0.0
scikit-image>=0.24.0,<0.25.0
tqdm>=4.66.0,<5.0.0
ftfy>=6.2.0,<7.0.0
tensorboard>=2.16.0,<3.0.0
easydict>=1.13.0,<2.0.0
clean-fid==0.1.35
torchmetrics>=1.4.0,<2.0.0
kornia>=0.7.0,<0.8.0
lpips>=0.1.4,<0.2.0
controlnet_aux>=0.0.7,<0.0.8
segment-anything>=1.0.0,<2.0.0
rembg[gpu]>=2.0.56,<3.0.0
moviepy>=1.0.3,<2.0.0
typer>=0.12.0,<0.13.0
rich>=13.7.0,<14.0.0
shellingham>=1.5.0,<2.0.0
"""

# 3. Criar o arquivo requirements.txt no ambiente do Colab
requirements_file_path = "/content/valentina_flux_requirements.txt"
with open(requirements_file_path, "w") as f:
    f.write(requirements_content)
print(f"📄 {requirements_file_path} criado com dependências do mflux.")

# 4. Instalar dependências usando uv pip install
print(f"🚀 Instalando dependências de {requirements_file_path} com uv...")
print("🕒 Isso pode levar alguns minutos...")
!uv pip install -q -r {requirements_file_path} --extra-index-url https://download.pytorch.org/whl/cu121 --index-strategy unsafe-best-match

print("✅ Dependências instaladas com sucesso usando uv!")
print("🎯 Ambiente otimizado para treinamento de LoRA FLUX")

# Verificação final de CUDA e PyTorch
import torch
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🎮 CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name()}")
    print(f"💾 VRAM total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    print(f"🧹 VRAM livre: {torch.cuda.memory_allocated() / 1024**3:.1f}GB")
else:
    print("⚠️ CUDA não detectado - usando CPU (MUITO LENTO)")

# Limpeza de memória inicial
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
print("✅ Verificação de ambiente concluída!")

In [ ]:
# 🖥️ Verificação e Otimização da GPU
import torch
import subprocess

print("🔍 Verificando configuração da GPU...")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader,nounits

# Detectar capacidades da GPU
gpu_name = subprocess.check_output(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"]).decode().strip()
print(f"📱 GPU detectada: {gpu_name}")

# Ajustar configurações baseado na GPU
if "T4" in gpu_name:
    print("🔧 Otimizações para Tesla T4")
    MIXED_PRECISION = "fp16"  # T4 funciona melhor com fp16
    TRAIN_BATCH_SIZE = 1
    GRADIENT_ACCUMULATION_STEPS = 6
elif "V100" in gpu_name:
    print("🔧 Otimizações para V100")
    MIXED_PRECISION = "fp16"
    TRAIN_BATCH_SIZE = 1
    GRADIENT_ACCUMULATION_STEPS = 8
elif "A100" in gpu_name or "H100" in gpu_name:
    print("🔧 Otimizações para GPU high-end")
    MIXED_PRECISION = "bf16"  # Melhor qualidade
    TRAIN_BATCH_SIZE = 2
    GRADIENT_ACCUMULATION_STEPS = 4
else:
    print("🔧 Configurações padrão")
    MIXED_PRECISION = "fp16"

print(f"✅ Configurações ajustadas: {MIXED_PRECISION}, batch={TRAIN_BATCH_SIZE}")

# Limpar cache da GPU
torch.cuda.empty_cache()
print(f"🧹 Cache da GPU limpo. Memória livre: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

print("Instalando dependências... Por favor, aguarde.")
!pip install -q diffusers transformers accelerate bitsandbytes safetensors peft xformers huggingface_hub torch torchvision torchaudio --upgrade

print("Dependências instaladas.")

In [ ]:
from huggingface_hub import notebook_login

print("Por favor, faça login na sua conta Hugging Face para baixar os modelos.")
notebook_login()

## Célula 4: Criação da Estrutura de Diretórios no Colab

In [ ]:
# 📁 Criação da Estrutura de Diretórios
import os
from pathlib import Path

# Criar todos os diretórios necessários
directories = [
    COLAB_MODELS_PATH,
    COLAB_DATASET_PATH,
    COLAB_OUTPUT_PATH,
    COLAB_LOGS_PATH,
    f"{COLAB_DATASET_PATH}/instance_images",
    f"{COLAB_DATASET_PATH}/class_images",
    f"{COLAB_OUTPUT_PATH}/checkpoints",
    f"{COLAB_OUTPUT_PATH}/samples"
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"📂 Criado: {directory}")

print("\n✅ Estrutura de diretórios criada com sucesso!")

## Célula 5: Upload e Preparação do Dataset de Identidade

In [ ]:
# 📤 Upload e Preparação do Dataset de Identidade
from google.colab import files
import zipfile
import shutil
from PIL import Image
import os

print("📥 === UPLOAD DO DATASET DE IDENTIDADE ===")
print(f"Por favor, faça upload do arquivo: {DATASET_ZIP_FILENAME}")
print("🧬 Dataset gerado pelo valentina_dataset_generator_colab.ipynb")
print("📊 Contém 18 imagens com seeds sequenciais para máxima consistência facial")
uploaded_dataset = files.upload()

if DATASET_ZIP_FILENAME in uploaded_dataset:
    print(f"📦 Extraindo {DATASET_ZIP_FILENAME}...")
    with zipfile.ZipFile(DATASET_ZIP_FILENAME, 'r') as zip_ref:
        zip_ref.extractall(COLAB_DATASET_PATH)
    
    # Encontrar as imagens extraídas
    instance_images_path = f"{COLAB_DATASET_PATH}/instance_images"
    
    # Verificar estrutura do dataset
    extracted_files = []
    for root, dirs, files in os.walk(COLAB_DATASET_PATH):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                extracted_files.append(os.path.join(root, file))
    
    print(f"🖼️ Encontradas {len(extracted_files)} imagens de identidade")
    
    # Mover imagens para pasta de instância se necessário
    if not os.path.exists(instance_images_path):
        os.makedirs(instance_images_path, exist_ok=True)
    
    processed_count = 0
    for i, img_path in enumerate(extracted_files):
        new_path = f"{instance_images_path}/valentina_{i:03d}.png"
        
        # Verificar se já não está na pasta correta
        if os.path.dirname(img_path) == instance_images_path:
            continue
            
        # Converter e redimensionar se necessário
        with Image.open(img_path) as img:
            # Converter para RGB se necessário
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Verificar se já está no tamanho correto
            if img.size == (RESOLUTION, RESOLUTION):
                # Salvar direto se já está no tamanho certo
                img.save(new_path, 'PNG', quality=95)
            else:
                # Redimensionar mantendo aspect ratio
                img.thumbnail((RESOLUTION, RESOLUTION), Image.Resampling.LANCZOS)
                
                # Criar imagem quadrada com padding
                new_img = Image.new('RGB', (RESOLUTION, RESOLUTION), (255, 255, 255))
                paste_x = (RESOLUTION - img.width) // 2
                paste_y = (RESOLUTION - img.height) // 2
                new_img.paste(img, (paste_x, paste_y))
                
                new_img.save(new_path, 'PNG', quality=95)
        
        processed_count += 1
        print(f"✅ Processada: {os.path.basename(img_path)} -> {os.path.basename(new_path)}")
    
    print(f"\n📊 Dataset de identidade preparado: {len(extracted_files)} imagens")
    print(f"🎯 Imagens processadas: {processed_count}")
    !ls -la {instance_images_path}
    
    # Verificar se existe metadata do dataset
    metadata_path = f"{COLAB_DATASET_PATH}/dataset_metadata.json"
    if os.path.exists(metadata_path):
        print(f"📋 Metadata do dataset encontrado: {metadata_path}")
        with open(metadata_path, 'r') as f:
            import json
            metadata = json.load(f)
            if isinstance(metadata, list) and len(metadata) > 0:
                first_meta = metadata[0]
                print(f"🧬 Configuração do dataset:")
                print(f"   • Arquitetura: {first_meta.get('architecture', 'N/A')}")
                print(f"   • Foco: {first_meta.get('optimization_focus', 'N/A')}")
                print(f"   • Base Model: {first_meta.get('base_model', 'N/A')}")
                print(f"   • Seeds: {metadata[0].get('seed', 'N/A')} a {metadata[-1].get('seed', 'N/A')}")
else:
    print(f"❌ ERRO: {DATASET_ZIP_FILENAME} não encontrado!")
    print("🔍 Certifique-se de que:")
    print("1. Gerou o dataset usando valentina_dataset_generator_colab.ipynb")
    print("2. Baixou o arquivo valentina_identity_4lora_dataset_flux.zip")
    print("3. Está fazendo upload do arquivo correto")
    raise FileNotFoundError("Dataset de identidade não foi enviado")

print("\n✅ Dataset de identidade preparado com sucesso!")
print("🧬 Pronto para treinar LoRA focada em identidade facial")
print(f"🎭 Trigger word que será treinada: '{INSTANCE_PROMPT}'")

## Célula 6: Preparação do Modelo Base para Treinamento de Identidade
Carrega o modelo FLUX.1-dev oficial e o prepara para treinamento da LoRA de identidade facial.

In [ ]:
import torch
from diffusers import FluxPipeline
import gc
import os
from pathlib import Path

print("🚀 Iniciando preparação do modelo base para identidade...")

# === VALIDAÇÃO LOCAL DO DATASET (antes do upload) ===
# 🔍 Verificação se o dataset foi criado corretamente ANTES do upload
local_dataset_path = Path("valentina_identity_4lora_dataset_flux")
if local_dataset_path.exists():
    local_images = list(local_dataset_path.glob("*.png")) + list(local_dataset_path.glob("*.jpg"))
    if len(local_images) >= 15:
        print(f"✅ Dataset local válido: {len(local_images)} imagens encontradas")
        print("📦 Pronto para fazer upload do dataset para o Colab")
    else:
        print(f"⚠️ Dataset incompleto: apenas {len(local_images)} imagens (mínimo: 15)")
        print("🔄 Execute novamente o valentina_dataset_generator_colab.ipynb")
else:
    print("❌ Pasta valentina_identity_4lora_dataset_flux não encontrada")
    print("🔄 Execute o valentina_dataset_generator_colab.ipynb primeiro")

# Configurar dtype baseado na GPU
if torch.cuda.is_available():
    if torch.cuda.get_device_capability()[0] >= 8:  # Ampere+
        model_dtype = torch.bfloat16
        print("💎 Usando bfloat16 (GPU Ampere+)")
    else:
        model_dtype = torch.float16
        print("⚡ Usando float16 (GPU older)")
else:
    model_dtype = torch.float32
    print("🐌 Usando float32 (CPU)")

# Para o treinamento, o mais simples é usar o modelo ID diretamente
# O script de treinamento irá carregar o modelo automaticamente
model_path = BASE_MODEL_ID
print(f"🎯 Modelo configurado para treinamento: {model_path}")
print("🧬 Foco: Treinamento de identidade facial pura")

# Verificar disponibilidade do modelo
print(f"✅ Modelo base preparado: {BASE_MODEL_ID}")
print("📝 O script de treinamento carregará o modelo automaticamente")

# Alternativamente, se precisar baixar o modelo localmente:
try:
    print(f"\n📦 Verificando disponibilidade do modelo: {BASE_MODEL_ID}")
    
    # Teste rápido de carregamento para verificar acesso
    pipeline = FluxPipeline.from_pretrained(
        BASE_MODEL_ID,
        torch_dtype=model_dtype,
        use_safetensors=True,
        low_cpu_mem_usage=True
    )
    
    print("✅ Modelo acessível e compatível!")
    
    # Salvar localmente se necessário
    model_save_path = f"{COLAB_MODELS_PATH}/flux_base_for_identity"
    print(f"💾 Salvando modelo base em: {model_save_path}")
    
    pipeline.save_pretrained(
        model_save_path,
        safe_serialization=True
    )
    
    # Atualizar caminho para apontar para o modelo local
    model_path = model_save_path
    print(f"🎯 Modelo salvo localmente: {model_path}")
    
    # Verificar componentes do modelo
    print("\n🔍 Componentes do modelo verificados:")
    if hasattr(pipeline, 'transformer'):
        print("   ✅ Transformer (componente principal)")
    if hasattr(pipeline, 'text_encoder'):
        print("   ✅ Text Encoder") 
    if hasattr(pipeline, 'text_encoder_2'):
        print("   ✅ Text Encoder 2")
    if hasattr(pipeline, 'vae'):
        print("   ✅ VAE")
    
    print(f"\n🎯 Modelo preparado: {model_path}")
    print("🧬 Foco: Treinamento de identidade facial pura (sem LoRAs adicionais)")
    
except Exception as e:
    print(f"⚠️ Não foi possível baixar o modelo localmente: {e}")
    print("🔄 Usando modelo ID diretamente para o treinamento")
    model_path = BASE_MODEL_ID
    print(f"🎯 Modelo para treinamento: {model_path}")

finally:
    # Limpar memória
    if 'pipeline' in locals():
        del pipeline
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 Memória limpa após verificação do modelo")

print(f"\n✅ Preparação concluída!")
print(f"🎭 Modelo pronto para treinamento de identidade: {model_path}")
print("🧬 Próximo: Iniciar treinamento da LoRA de identidade facial")

## Célula 7: Treinamento da LoRA de Identidade Facial

In [ ]:
# 🚀 CÉLULA 7: TREINAMENTO FLUX LORA DE IDENTIDADE FACIAL
# Sistema completo otimizado para Google Colab

import os
import sys
import torch
import gc
import json
import time
import random
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import T5EncoderModel, T5Tokenizer, CLIPTextModel, CLIPTokenizer
from diffusers import FluxPipeline, FluxTransformer2DModel
from diffusers.models.attention_processor import LoRAAttnProcessor2_0
from diffusers.loaders import AttnProcsLayers
from diffusers.optimization import get_scheduler
import safetensors.torch as st

print("🚀 INICIANDO TREINAMENTO FLUX LORA DE IDENTIDADE FACIAL")
print("=" * 60)

# Configurar seeds para reprodutibilidade
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(SEED)

# Verificações iniciais
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16

print(f"📱 Device: {device}")
print(f"🔢 Dtype: {dtype}")
print(f"💾 VRAM disponível: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

# Dataset personalizado para FLUX
class FluxLoRADataset(Dataset):
    def __init__(self, dataset_path, instance_prompt, class_prompt, tokenizer_1, tokenizer_2, size=1024):
        self.dataset_path = Path(dataset_path)
        self.instance_prompt = instance_prompt
        self.class_prompt = class_prompt
        self.tokenizer_1 = tokenizer_1
        self.tokenizer_2 = tokenizer_2
        self.size = size
        
        # Encontrar imagens de instância
        instance_path = self.dataset_path / "instance_images"
        if not instance_path.exists():
            instance_path = self.dataset_path
        
        self.instance_images = []
        for ext in ["*.jpg", "*.jpeg", "*.png"]:
            self.instance_images.extend(list(instance_path.glob(ext)))
        
        print(f"📊 Imagens de instância encontradas: {len(self.instance_images)}")
        
        if len(self.instance_images) == 0:
            raise ValueError("Nenhuma imagem de instância encontrada!")
    
    def __len__(self):
        return len(self.instance_images)
    
    def __getitem__(self, idx):
        # Carregar e processar imagem
        image_path = self.instance_images[idx]
        image = Image.open(image_path).convert("RGB")
        
        # Redimensionar mantendo aspect ratio
        image = image.resize((self.size, self.size), Image.Resampling.LANCZOS)
        
        # Converter para tensor
        image = torch.from_numpy(np.array(image)).float() / 255.0
        image = image.permute(2, 0, 1)  # HWC -> CHW
        
        # Normalizar para [-1, 1]
        image = (image - 0.5) * 2.0
        
        # Tokenizar prompts
        instance_tokens_1 = self.tokenizer_1(
            self.instance_prompt,
            truncation=True,
            padding="max_length",
            max_length=77,
            return_tensors="pt"
        ).input_ids[0]
        
        instance_tokens_2 = self.tokenizer_2(
            self.instance_prompt,
            truncation=True,
            padding="max_length",
            max_length=256,
            return_tensors="pt"
        ).input_ids[0]
        
        class_tokens_1 = self.tokenizer_1(
            self.class_prompt,
            truncation=True,
            padding="max_length",
            max_length=77,
            return_tensors="pt"
        ).input_ids[0]
        
        class_tokens_2 = self.tokenizer_2(
            self.class_prompt,
            truncation=True,
            padding="max_length",
            max_length=256,
            return_tensors="pt"
        ).input_ids[0]
        
        return {
            "pixel_values": image,
            "instance_prompt_ids_1": instance_tokens_1,
            "instance_prompt_ids_2": instance_tokens_2,
            "class_prompt_ids_1": class_tokens_1,
            "class_prompt_ids_2": class_tokens_2
        }

# Função para aplicar LoRA no transformer
def setup_lora_layers(transformer):
    """Aplicar LoRA nos módulos de atenção do transformer FLUX (compatível FLUX.1-dev Colab)"""
    lora_attn_procs = {}
    for name, module in transformer.named_modules():
        if "attn" in name and hasattr(module, "to_k"):
            # Use only the processor key as required by set_attn_processor
            lora_attn_procs[f"{name}.processor"] = LoRAAttnProcessor2_0()
    transformer.set_attn_processor(lora_attn_procs)
    return lora_attn_procs

# Função principal de treinamento
def train_flux_lora():
    """Executar treinamento FLUX LoRA completo"""
    try:
        print("\n📥 FASE 1: Carregando modelo base...")
        
        # Carregar pipeline FLUX
        pipe = FluxPipeline.from_pretrained(
            BASE_MODEL_ID,
            torch_dtype=dtype,
            use_safetensors=True,
            variant="fp16" if dtype == torch.float16 else None
        )
        
        pipe = pipe.to(device)
        print(f"✅ Pipeline FLUX carregado: {BASE_MODEL_ID}")
        
        # Extrair componentes
        transformer = pipe.transformer
        vae = pipe.vae
        text_encoder = pipe.text_encoder
        text_encoder_2 = pipe.text_encoder_2
        tokenizer_1 = pipe.tokenizer
        tokenizer_2 = pipe.tokenizer_2
        scheduler = pipe.scheduler
        
        print("✅ Componentes extraídos com sucesso")
        
        print("\n📥 FASE 2: Configurando LoRA...")
        
        # Configurar LoRA
        lora_attn_procs = setup_lora_layers(transformer)
        # Debug: inspecionar atributos dos processadores
        print('--- DEBUG: Inspecionando attn_processors ---')
        for name, proc in getattr(transformer, 'attn_processors', {}).items():
            print(f'Processador: {name} | Tipo: {type(proc)}')
            print('Atributos:', dir(proc))
        print('--- FIM DEBUG ---')
        # Obter parâmetros treináveis corretamente dos LoRAAttnProcessor2_0
        trainable_params = []
        for proc in getattr(transformer, 'attn_processors', {}).values():
            for attr in ['to_q_lora', 'to_k_lora', 'to_v_lora', 'to_out_lora']:
                lora_layer = getattr(proc, attr, None)
                if lora_layer is not None:
                    for p in lora_layer.parameters():
                        if p.requires_grad:
                            trainable_params.append(p)
        print(f"✅ LoRA configurado: {len(trainable_params)} parâmetros treináveis")
        print(f"🔧 Rank: {LORA_RANK}, Alpha: {LORA_ALPHA}, Dropout: {LORA_DROPOUT}")
        
        print("\n📥 FASE 3: Preparando dataset...")
        
        # Criar dataset
        dataset = FluxLoRADataset(
            COLAB_DATASET_PATH,
            INSTANCE_PROMPT,
            CLASS_PROMPT,
            tokenizer_1,
            tokenizer_2,
            size=RESOLUTION
        )
        
        # Criar dataloader
        dataloader = DataLoader(
            dataset,
            batch_size=TRAIN_BATCH_SIZE,
            shuffle=True,
            num_workers=0,
            pin_memory=True
        )
        
        print(f"✅ Dataset preparado: {len(dataset)} imagens")
        
        print("\n📥 FASE 4: Configurando otimizador...")
        
        # Configurar otimizador
        if USE_8BIT_ADAM:
            try:
                import bitsandbytes as bnb
                optimizer_cls = bnb.optim.AdamW8bit
                print("✅ Usando AdamW8bit")
            except ImportError:
                optimizer_cls = torch.optim.AdamW
                print("⚠️ Fallback para AdamW padrão")
        else:
            optimizer_cls = torch.optim.AdamW
        
        optimizer = optimizer_cls(
            trainable_params,
            lr=LEARNING_RATE,
            betas=(ADAM_BETA1, ADAM_BETA2),
            weight_decay=ADAM_WEIGHT_DECAY,
            eps=ADAM_EPSILON
        )
        
        # Configurar scheduler
        lr_scheduler = get_scheduler(
            LR_SCHEDULER,
            optimizer=optimizer,
            num_warmup_steps=LR_WARMUP_STEPS,
            num_training_steps=MAX_TRAIN_STEPS,
            num_cycles=LR_NUM_CYCLES
        )
        
        print(f"✅ Otimizador configurado: {optimizer_cls.__name__}")
        print(f"📈 Scheduler: {LR_SCHEDULER}")
        
        print("\n📥 FASE 5: Iniciando treinamento...")
        
        # Preparar modelos para treinamento
        transformer.train()
        vae.eval()
        text_encoder.eval()
        text_encoder_2.eval()
        
        # Desabilitar gradientes nos modelos não treináveis
        vae.requires_grad_(False)
        text_encoder.requires_grad_(False)
        text_encoder_2.requires_grad_(False)
        
        # Variáveis de controle
        global_step = 0
        training_stats = []
        
        # Progress bar
        progress_bar = tqdm(total=MAX_TRAIN_STEPS, desc="Treinamento")
        
        # Loop de treinamento
        epoch = 0
        while global_step < MAX_TRAIN_STEPS:
            epoch += 1
            epoch_loss = 0
            
            for batch_idx, batch in enumerate(dataloader):
                if global_step >= MAX_TRAIN_STEPS:
                    break
                
                # Mover batch para device
                pixel_values = batch["pixel_values"].to(device, dtype=dtype)
                instance_prompt_ids_1 = batch["instance_prompt_ids_1"].to(device)
                instance_prompt_ids_2 = batch["instance_prompt_ids_2"].to(device)
                class_prompt_ids_1 = batch["class_prompt_ids_1"].to(device)
                class_prompt_ids_2 = batch["class_prompt_ids_2"].to(device)
                
                # Encode imagens com VAE
                with torch.no_grad():
                    latents = vae.encode(pixel_values).latent_dist.sample()
                    latents = latents * vae.config.scaling_factor
                
                # Adicionar ruído
                noise = torch.randn_like(latents)
                timesteps = torch.randint(
                    0, scheduler.config.num_train_timesteps,
                    (latents.shape[0],), device=device
                ).long()
                
                noisy_latents = scheduler.add_noise(latents, noise, timesteps)
                
                # Encode prompts
                with torch.no_grad():
                    # Text encoder 1 (CLIP)
                    encoder_hidden_states_1 = text_encoder(instance_prompt_ids_1)[0]
                    class_hidden_states_1 = text_encoder(class_prompt_ids_1)[0]
                    
                    # Text encoder 2 (T5)
                    encoder_hidden_states_2 = text_encoder_2(instance_prompt_ids_2)[0]
                    class_hidden_states_2 = text_encoder_2(class_prompt_ids_2)[0]
                
                # Prediction do transformer
                model_pred = transformer(
                    noisy_latents,
                    timesteps,
                    encoder_hidden_states=encoder_hidden_states_1,
                    pooled_projections=encoder_hidden_states_2.mean(dim=1),
                    return_dict=False
                )[0]
                
                # Calcular loss
                if scheduler.config.prediction_type == "epsilon":
                    target = noise
                elif scheduler.config.prediction_type == "v_prediction":
                    target = scheduler.get_velocity(latents, noise, timesteps)
                else:
                    raise ValueError(f"Prediction type {scheduler.config.prediction_type} não suportado")
                
                loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
                
                # Prior preservation loss
                if PRIOR_LOSS_WEIGHT > 0:
                    with torch.no_grad():
                        class_model_pred = transformer(
                            noisy_latents,
                            timesteps,
                            encoder_hidden_states=class_hidden_states_1,
                            pooled_projections=class_hidden_states_2.mean(dim=1),
                            return_dict=False
                        )[0]
                    
                    prior_loss = F.mse_loss(class_model_pred.float(), target.float(), reduction="mean")
                    loss = loss + PRIOR_LOSS_WEIGHT * prior_loss
                
                # Backpropagation
                loss.backward()
                
                if (global_step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                    # Gradient clipping
                    if MAX_GRAD_NORM > 0:
                        torch.nn.utils.clip_grad_norm_(trainable_params, MAX_GRAD_NORM)
                    
                    optimizer.step()
                    lr_scheduler.step()
                    optimizer.zero_grad()
                
                # Logging
                current_loss = loss.detach().item()
                epoch_loss += current_loss
                
                # Update progress
                progress_bar.set_postfix({
                    "loss": f"{current_loss:.4f}",
                    "lr": f"{lr_scheduler.get_last_lr()[0]:.2e}",
                    "epoch": epoch
                })
                progress_bar.update(1)
                
                # Save checkpoint
                if global_step > 0 and global_step % SAVE_STEPS == 0:
                    save_checkpoint(transformer, lora_layers, global_step)
                
                # Armazenar estatísticas
                if global_step % 10 == 0:
                    training_stats.append({
                        "step": global_step,
                        "loss": current_loss,
                        "lr": lr_scheduler.get_last_lr()[0],
                        "epoch": epoch
                    })
                
                global_step += 1
                
                # Limpeza de memória
                if global_step % 50 == 0:
                    torch.cuda.empty_cache()
        
        progress_bar.close()
        
        print("\n📥 FASE 6: Salvando modelo final...")
        
        # Salvar LoRA final
        save_final_lora(lora_layers, training_stats)
        
        print("\n🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
        return True
        
    except Exception as e:
        print(f"\n❌ ERRO DURANTE TREINAMENTO: {e}")
        import traceback
        traceback.print_exc()
        return False
    
    finally:
        # Limpeza
        torch.cuda.empty_cache()
        gc.collect()

def save_checkpoint(transformer, lora_layers, step):
    """Salvar checkpoint do treinamento"""
    checkpoint_dir = f"{COLAB_OUTPUT_PATH}/checkpoint-{step}"
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Salvar LoRA weights
    lora_state_dict = lora_layers.state_dict()
    checkpoint_path = f"{checkpoint_dir}/pytorch_lora_weights.safetensors"
    st.save_file(lora_state_dict, checkpoint_path)
    
    print(f"✅ Checkpoint salvo: {checkpoint_path}")

def save_final_lora(lora_layers, training_stats):
    """Salvar LoRA final e metadados"""
    os.makedirs(COLAB_OUTPUT_PATH, exist_ok=True)
    
    # Salvar LoRA weights
    lora_state_dict = lora_layers.state_dict()
    lora_path = f"{COLAB_OUTPUT_PATH}/valentina_identity_lora.safetensors"
    st.save_file(lora_state_dict, lora_path)
    
    # Salvar metadados
    metadata = {
        "model_name": "Valentina Identity LoRA",
        "base_model": BASE_MODEL_ID,
        "training_config": {
            "lora_rank": LORA_RANK,
            "lora_alpha": LORA_ALPHA,
            "lora_dropout": LORA_DROPOUT,
            "learning_rate": LEARNING_RATE,
            "max_train_steps": MAX_TRAIN_STEPS,
            "batch_size": TRAIN_BATCH_SIZE,
            "resolution": RESOLUTION,
            "instance_prompt": INSTANCE_PROMPT,
            "class_prompt": CLASS_PROMPT
        },
        "training_stats": training_stats[-100:],  # Últimas 100 entradas
        "timestamp": time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(f"{COLAB_OUTPUT_PATH}/training_metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✅ LoRA salvo: {lora_path}")
    print(f"📊 Tamanho: {os.path.getsize(lora_path) / 1024 / 1024:.1f}MB")

# Executar treinamento
if __name__ == "__main__":
    print(f"🎯 Configurações de treinamento:")
    print(f"   📊 Steps: {MAX_TRAIN_STEPS}")
    print(f"   🧠 LoRA Rank: {LORA_RANK}")
    print(f"   📈 Learning Rate: {LEARNING_RATE}")
    print(f"   🎭 Trigger: '{INSTANCE_PROMPT}'")
    print(f"   📐 Resolução: {RESOLUTION}x{RESOLUTION}")
    
    success = train_flux_lora()
    
    if success:
        print("\n✅ TREINAMENTO FINALIZADO COM SUCESSO!")
        print("🎉 LoRA de identidade facial da Valentina está pronta!")
    else:
        print("\n❌ TREINAMENTO FALHOU!")
        print("📋 Verifique os logs acima para detalhes do erro.")

## Célula 8: Processamento dos Resultados de Identidade

In [ ]:
# 📦 CÉLULA 8: PROCESSAMENTO E DOWNLOAD DOS RESULTADOS
# Análise, validação e empacotamento da LoRA de identidade facial

import os
import json
import shutil
import zipfile
from pathlib import Path
from google.colab import files
import torch
import safetensors.torch as st
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import FluxPipeline

print("📦 PROCESSAMENTO DOS RESULTADOS DE IDENTIDADE FACIAL")
print("=" * 60)

def analyze_training_results():
    """Analisar resultados do treinamento"""
    print("🔍 Analisando resultados do treinamento...")
    
    results = {
        "lora_file": None,
        "checkpoints": [],
        "metadata": None,
        "file_sizes": {},
        "training_completed": False
    }
    
    # Verificar arquivo principal da LoRA
    main_lora_path = f"{COLAB_OUTPUT_PATH}/valentina_identity_lora.safetensors"
    if os.path.exists(main_lora_path):
        results["lora_file"] = main_lora_path
        results["file_sizes"]["main_lora"] = os.path.getsize(main_lora_path)
        results["training_completed"] = True
        print(f"✅ LoRA principal encontrada: {os.path.basename(main_lora_path)}")
        print(f"   💾 Tamanho: {results['file_sizes']['main_lora'] / 1024 / 1024:.1f}MB")
    
    # Verificar checkpoints
    if os.path.exists(COLAB_OUTPUT_PATH):
        for item in os.listdir(COLAB_OUTPUT_PATH):
            if item.startswith("checkpoint-"):
                checkpoint_path = f"{COLAB_OUTPUT_PATH}/{item}"
                lora_checkpoint = f"{checkpoint_path}/pytorch_lora_weights.safetensors"
                if os.path.exists(lora_checkpoint):
                    results["checkpoints"].append({
                        "step": int(item.split("-")[1]),
                        "path": lora_checkpoint,
                        "size": os.path.getsize(lora_checkpoint)
                    })
    
    # Ordenar checkpoints por step
    results["checkpoints"].sort(key=lambda x: x["step"])
    print(f"📋 Checkpoints encontrados: {len(results['checkpoints'])}")
    
    # Verificar metadados
    metadata_path = f"{COLAB_OUTPUT_PATH}/training_metadata.json"
    if os.path.exists(metadata_path):
        with open(metadata_path, 'r') as f:
            results["metadata"] = json.load(f)
        print("✅ Metadados de treinamento encontrados")
    
    return results

def validate_lora_weights(lora_path):
    """Validar integridade dos pesos da LoRA"""
    print(f"🔍 Validando LoRA: {os.path.basename(lora_path)}")
    
    try:
        # Carregar e analisar pesos
        state_dict = st.load_file(lora_path)
        
        # Estatísticas dos pesos
        total_params = 0
        layer_info = {}
        
        for key, tensor in state_dict.items():
            total_params += tensor.numel()
            layer_type = key.split(".")[-2] if "." in key else "unknown"
            
            if layer_type not in layer_info:
                layer_info[layer_type] = {"count": 0, "params": 0}
            
            layer_info[layer_type]["count"] += 1
            layer_info[layer_type]["params"] += tensor.numel()
        
        print(f"   📊 Parâmetros totais: {total_params:,}")
        print(f"   🏗️ Camadas LoRA:")
        for layer_type, info in layer_info.items():
            print(f"      • {layer_type}: {info['count']} camadas, {info['params']:,} parâmetros")
        
        # Verificar se há pesos não-zero
        non_zero_weights = 0
        total_weights = 0
        
        for tensor in state_dict.values():
            non_zero_weights += (tensor != 0).sum().item()
            total_weights += tensor.numel()
        
        non_zero_ratio = non_zero_weights / total_weights if total_weights > 0 else 0
        print(f"   🎨 Pesos não-zero: {non_zero_ratio:.1%}")
        
        # Verificar range dos valores
        all_values = torch.cat([tensor.flatten() for tensor in state_dict.values()])
        print(f"   📊 Range de valores: [{all_values.min():.4f}, {all_values.max():.4f}]")
        print(f"   📊 Média: {all_values.mean():.4f}, Std: {all_values.std():.4f}")
        
        validation_result = {
            "total_params": total_params,
            "layer_info": layer_info,
            "non_zero_ratio": non_zero_ratio,
            "value_range": [float(all_values.min()), float(all_values.max())],
            "mean": float(all_values.mean()),
            "std": float(all_values.std()),
            "valid": True
        }
        
        print("✅ LoRA validada com sucesso!")
        return validation_result
        
    except Exception as e:
        print(f"❌ Erro na validação: {e}")
        return {"valid": False, "error": str(e)}

def create_test_generation(lora_path):
    """Criar geração de teste com a LoRA"""
    print("🎨 Criando geração de teste...")
    
    try:
        # Carregar pipeline
        pipe = FluxPipeline.from_pretrained(
            BASE_MODEL_ID,
            torch_dtype=torch.float16,
            use_safetensors=True
        )
        
        # Carregar LoRA
        pipe.load_lora_weights(lora_path)
        pipe = pipe.to("cuda")
        
        # Prompt de teste
        test_prompt = f"a professional portrait photo of {INSTANCE_PROMPT.replace('a photo of ', '')}, high quality, detailed face, studio lighting"
        
        print(f"   📝 Prompt: {test_prompt}")
        
        # Gerar imagem
        with torch.no_grad():
            image = pipe(
                prompt=test_prompt,
                height=1024,
                width=1024,
                num_inference_steps=25,
                guidance_scale=7.5,
                generator=torch.Generator(device="cuda").manual_seed(42)
            ).images[0]
        
        # Salvar imagem de teste
        test_image_path = f"{COLAB_OUTPUT_PATH}/test_generation.png"
        image.save(test_image_path)
        
        print(f"✅ Geração de teste salva: {test_image_path}")
        
        # Limpar memória
        del pipe
        torch.cuda.empty_cache()
        
        return test_image_path
        
    except Exception as e:
        print(f"⚠️ Erro na geração de teste: {e}")
        print("   📝 Isso pode ser normal se o Colab não tiver VRAM suficiente")
        return None

def create_documentation(results, validation_data):
    """Criar documentação completa da LoRA"""
    print("📝 Criando documentação...")
    
    # Calcular estatísticas do dataset
    instance_images_path = f"{COLAB_DATASET_PATH}/instance_images"
    if not os.path.exists(instance_images_path):
        instance_images_path = COLAB_DATASET_PATH
    
    image_files = []
    for ext in ["*.jpg", "*.jpeg", "*.png"]:
        image_files.extend(list(Path(instance_images_path).glob(ext)))
    
    # Criar README detalhado
    readme_content = f"""# Valentina Identity LoRA - FLUX.1-dev

## 🎯 Objetivo
Esta LoRA foi treinada especificamente para capturar e preservar a **identidade facial da Valentina**, focando em:
- Consistência de características faciais
- Preservação da identidade visual
- Geração de alta qualidade com FLUX.1-dev

## 📋 Informações Técnicas

### Modelo Base
- **Arquitetura**: FLUX.1-dev
- **Desenvolvedor**: Black Forest Labs
- **Tipo**: Modelo de difusão transformer

### Configurações de Treinamento
- **LoRA Rank**: {LORA_RANK}
- **LoRA Alpha**: {LORA_ALPHA}
- **LoRA Dropout**: {LORA_DROPOUT}
- **Learning Rate**: {LEARNING_RATE}
- **Steps de Treinamento**: {MAX_TRAIN_STEPS}
- **Batch Size**: {TRAIN_BATCH_SIZE}
- **Resolução**: {RESOLUTION}x{RESOLUTION}
- **Imagens de Treinamento**: {len(image_files)}

### Dataset
- **Fonte**: Gerado pelo valentina_dataset_generator_colab.ipynb
- **Tipo**: Imagens com seeds sequenciais para máxima consistência
- **Foco**: Identidade facial pura (não NSFW)
- **Características preservadas**:
  - Idade: 25 anos
  - Formato do rosto: Oval
  - Olhos: Amendoados castanho-escuros
  - Lábios: Carnudos com arco do cupido
  - Pele: Dourada mediterrânea
  - Cabelo: Castanho médio com reflexos dourados
  - Tatuagens: NENHUMA (pele limpa)

## 🚀 Como Usar

### mflux (macOS)
```bash
mflux-generate \\
    --model "/path/to/flux.1-dev" \\
    --lora "valentina_identity_lora.safetensors" \\
    --lora-scale 0.8 \\
    --prompt "a photo of vltna woman, [seu prompt]" \\
    --steps 25 \\
    --height 1024 \\
    --width 1024
```

### ComfyUI
1. Coloque o arquivo .safetensors na pasta `models/loras/`
2. Use o nó "Load LoRA" no workflow
3. Configure o peso entre 0.7-1.0
4. Use o trigger word `vltna woman` nos prompts

### Automatic1111
1. Coloque o arquivo na pasta `models/Lora/`
2. Use `<lora:valentina_identity_lora:0.8>` no prompt
3. Inclua `vltna woman` no prompt

## 🎨 Prompts Recomendados

### Para Identidade/Retratos
```
a photo of vltna woman, professional portrait, studio lighting, detailed face
vltna woman, natural smile, elegant pose, high quality photography
a photo of vltna woman, looking at camera, soft lighting, photorealistic
```

### Para Situações Específicas
```
vltna woman, business attire, office environment, confident expression
a photo of vltna woman, casual outfit, outdoor scene, natural lighting
vltna woman, elegant dress, formal event, sophisticated pose
```

## ⚙️ Configurações Recomendadas

| Parâmetro | Valor Recomendado | Observações |
|-----------|------------------|-------------|
| LoRA Weight | 0.7 - 1.0 | Comece com 0.8 |
| Steps | 25 - 35 | 25 é ideal para velocidade |
| CFG Scale | 7 - 9 | 7.5 é um bom ponto de partida |
| Resolução | 1024x1024 | Resolução de treinamento |
| Sampler | DPM++ 2M | Para FLUX.1-dev |

## 📋 Resultados de Validação

- **Parâmetros Totais**: {validation_data.get('total_params', 'N/A'):,}
- **Pesos Não-Zero**: {validation_data.get('non_zero_ratio', 0):.1%}
- **Range de Valores**: [{validation_data.get('value_range', [0, 0])[0]:.4f}, {validation_data.get('value_range', [0, 0])[1]:.4f}]
- **Média dos Pesos**: {validation_data.get('mean', 0):.4f}

## ⚠️ Notas Importantes

1. **Foco em Identidade**: Esta LoRA foi treinada especificamente para preservar a identidade facial da Valentina
2. **Não NSFW**: Otimizada para geração de conteúdo não explícito
3. **Consistência**: Use seeds semelhantes para manter consistência entre gerações
4. **Qualidade**: Melhor qualidade com resoluções altas (1024x1024 ou superior)
5. **Trigger Word**: Sempre inclua `vltna woman` para ativar a identidade

## 📊 Performance

- **VRAM Necessária**: ~8-12GB para geração 1024x1024
- **Tempo de Geração**: ~30-60 segundos (dependendo da GPU)
- **Qualidade**: Alta fidelidade à identidade da Valentina
- **Estabilidade**: Testada em múltiplas gerações

## 🔄 Atualizações

- **Versão**: 1.0 (Treinamento inicial de identidade)
- **Data**: {results.get('metadata', {}).get('timestamp', 'N/A')}
- **Status**: Pronta para uso em produção

## 📞 Suporte

Para problemas ou dúvidas:
1. Verifique se está usando o modelo base correto (FLUX.1-dev)
2. Confirme que o trigger word `vltna woman` está no prompt
3. Ajuste o peso da LoRA entre 0.7-1.0
4. Use as configurações recomendadas acima

---

**Gerado pelo Sistema de Treinamento Valentina LoRA**  
**Dataset**: valentina_identity_4lora_dataset_flux  
**Notebook**: valentina_colab_facial_lora_trainer_flux.ipynb
"""
    
    return readme_content

def create_final_package(results, validation_data):
    """Criar pacote final para download"""
    print("📦 Criando pacote final...")
    
    # Criar diretório do pacote
    package_dir = f"{COLAB_OUTPUT_PATH}/valentina_identity_lora_package"
    os.makedirs(package_dir, exist_ok=True)
    
    # Copiar LoRA principal
    if results["lora_file"]:
        final_lora_name = "valentina_identity_lora.safetensors"
        shutil.copy2(results["lora_file"], f"{package_dir}/{final_lora_name}")
        print(f"✅ LoRA copiada: {final_lora_name}")
    
    # Criar documentação
    readme_content = create_documentation(results, validation_data)
    with open(f"{package_dir}/README.md", 'w', encoding='utf-8') as f:
        f.write(readme_content)
    
    # Criar arquivo de configuração JSON
    config = {
        "model_info": {
            "name": "Valentina Identity LoRA",
            "version": "1.0",
            "base_model": BASE_MODEL_ID,
            "type": "identity_lora",
            "architecture": "flux_transformer"
        },
        "training_config": {
            "lora_rank": LORA_RANK,
            "lora_alpha": LORA_ALPHA,
            "lora_dropout": LORA_DROPOUT,
            "learning_rate": LEARNING_RATE,
            "max_steps": MAX_TRAIN_STEPS,
            "batch_size": TRAIN_BATCH_SIZE,
            "resolution": RESOLUTION,
            "gradient_accumulation_steps": GRADIENT_ACCUMULATION_STEPS
        },
        "character_profile": {
            "name": "Valentina Moreau",
            "age": 25,
            "face_shape": "oval",
            "eyes": "amendoados castanho-escuros",
            "lips": "carnudos com arco do cupido",
            "skin": "dourada mediterrânea",
            "hair": "castanho médio com reflexos dourados",
            "body_type": "curvilíneo",
            "tattoos": "nenhuma - pele limpa"
        },
        "usage": {
            "trigger_word": INSTANCE_PROMPT,
            "recommended_weight": "0.7-1.0",
            "compatible_software": ["mflux", "ComfyUI", "Automatic1111"],
            "optimal_resolution": "1024x1024",
            "recommended_steps": "25-35"
        },
        "validation": validation_data,
        "created_at": results.get("metadata", {}).get("timestamp", "unknown")
    }
    
    with open(f"{package_dir}/config.json", 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)
    
    # Copiar imagem de teste se existir
    test_image_path = f"{COLAB_OUTPUT_PATH}/test_generation.png"
    if os.path.exists(test_image_path):
        shutil.copy2(test_image_path, f"{package_dir}/sample_generation.png")
        print("✅ Imagem de teste incluída")
    
    # Copiar metadados de treinamento se existir
    metadata_path = f"{COLAB_OUTPUT_PATH}/training_metadata.json"
    if os.path.exists(metadata_path):
        shutil.copy2(metadata_path, f"{package_dir}/training_metadata.json")
    
    # Criar arquivo ZIP
    zip_filename = f"{COLAB_OUTPUT_PATH}/valentina_identity_lora_final.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
        for root, dirs, files in os.walk(package_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arc_name = os.path.relpath(file_path, package_dir)
                zipf.write(file_path, arc_name)
    
    package_size = os.path.getsize(zip_filename) / 1024 / 1024
    print(f"✅ Pacote criado: {zip_filename}")
    print(f"💾 Tamanho do pacote: {package_size:.1f}MB")
    
    return zip_filename, package_size

def display_summary(results, validation_data, package_info):
    """Exibir resumo final"""
    print("\n" + "=" * 60)
    print("🎉 RESUMO FINAL DO TREINAMENTO")
    print("=" * 60)
    
    if results["training_completed"]:
        print("✅ STATUS: TREINAMENTO CONCLUÍDO COM SUCESSO")
    else:
        print("⚠️ STATUS: TREINAMENTO INCOMPLETO OU COM PROBLEMAS")
        return
    
    print(f"\n🎯 MODELO:")
    print(f"   • Nome: Valentina Identity LoRA")
    print(f"   • Base: {BASE_MODEL_ID}")
    print(f"   • Tipo: Identidade Facial (não NSFW)")
    print(f"   • Trigger: '{INSTANCE_PROMPT}'")
    
    print(f"\n📊 ESTATÍSTICAS:")
    if validation_data.get('valid'):
        print(f"   • Parâmetros LoRA: {validation_data['total_params']:,}")
        print(f"   • Pesos ativos: {validation_data['non_zero_ratio']:.1%}")
        print(f"   • Tamanho do arquivo: {results['file_sizes']['main_lora'] / 1024 / 1024:.1f}MB")
    
    print(f"\n🔧 CONFIGURAÇÕES:")
    print(f"   • LoRA Rank: {LORA_RANK}")
    print(f"   • LoRA Alpha: {LORA_ALPHA}")
    print(f"   • Steps: {MAX_TRAIN_STEPS}")
    print(f"   • Learning Rate: {LEARNING_RATE}")
    print(f"   • Resolução: {RESOLUTION}x{RESOLUTION}")
    
    if package_info:
        zip_file, size = package_info
        print(f"\n📦 PACOTE FINAL:")
        print(f"   • Arquivo: {os.path.basename(zip_file)}")
        print(f"   • Tamanho: {size:.1f}MB")
        print(f"   • Conteúdo: LoRA + Documentação + Config")
    
    print(f"\n🚀 COMO USAR:")
    print(f"   1. Baixe o arquivo ZIP")
    print(f"   2. Extraia e use o arquivo .safetensors")
    print(f"   3. Configure peso 0.7-1.0 no seu software")
    print(f"   4. Use '{INSTANCE_PROMPT}' nos prompts")
    print(f"   5. Leia o README.md para detalhes")
    
    print(f"\n🎨 CARACTERÍSTICAS PRESERVADAS:")
    print(f"   • Idade: 25 anos")
    print(f"   • Rosto: Oval")
    print(f"   • Olhos: Amendoados castanho-escuros")
    print(f"   • Pele: Dourada mediterrânea, sem tatuagens")
    print(f"   • Cabelo: Castanho médio com reflexos")
    
    print("\n" + "=" * 60)

# Execução principal
if __name__ == "__main__":
    # Analisar resultados
    results = analyze_training_results()
    
    if not results["training_completed"]:
        print("❌ ERRO: Treinamento não foi concluído ou LoRA não foi encontrada!")
        print("\n🔍 Verificando diretório de saída:")
        if os.path.exists(COLAB_OUTPUT_PATH):
            for item in os.listdir(COLAB_OUTPUT_PATH):
                item_path = os.path.join(COLAB_OUTPUT_PATH, item)
                if os.path.isfile(item_path):
                    size = os.path.getsize(item_path) / 1024 / 1024
                    print(f"   📄 {item} ({size:.1f}MB)")
                else:
                    print(f"   📁 {item}/")
        else:
            print(f"   ❌ Diretório não existe: {COLAB_OUTPUT_PATH}")
    else:
        # Validar LoRA
        validation_data = validate_lora_weights(results["lora_file"])
        
        # Criar geração de teste (opcional)
        test_image = create_test_generation(results["lora_file"])
        
        # Criar pacote final
        package_info = create_final_package(results, validation_data)
        
        # Exibir resumo
        display_summary(results, validation_data, package_info)
        
        # Download do pacote
        if package_info:
            zip_file, _ = package_info
            print(f"\n📥 INICIANDO DOWNLOAD...")
            files.download(zip_file)
            print(f"✅ Download concluído: {os.path.basename(zip_file)}")
            
            print(f"\n🎉 PARABÉNS!")
            print(f"Sua LoRA de identidade facial da Valentina está pronta para uso!")
            print(f"\nPróximos passos:")
            print(f"1. Extraia o arquivo ZIP baixado")
            print(f"2. Leia o README.md para instruções detalhadas")
            print(f"3. Teste com o prompt: 'a photo of vltna woman, portrait'")
            print(f"4. Ajuste o peso da LoRA conforme necessário (0.7-1.0)")

## Célula 9: Limpeza (Opcional)
Descomente para remover arquivos grandes e economizar espaço no Colab se for continuar usando o runtime após o treinamento de identidade.

In [ ]:
# print("Limpando arquivos de treinamento de identidade...")
# !rm -rf {COLAB_MODELS_PATH}/{BASE_MODEL_ID.split('/')[-1]} # Remove o cache do modelo base
# !rm -f {DATASET_ZIP_FILENAME} # Remove o dataset ZIP
# !rm -rf {COLAB_DATASET_PATH}/* # Limpa imagens extraídas do dataset de identidade
# print("Limpeza concluída (arquivos de modelo e dataset de identidade).")
# torch.cuda.empty_cache()